In [1]:
import tensorflow as tf
from PIL import Image
import numpy as np
import time
from PIL import Image, ImageDraw

In [3]:
path = "./multi_person_mobilenet_v1_075_float.tflite"

interpreter = tf.lite.Interpreter(path)

input_details = interpreter.get_input_details()
print(str(input_details))
output_details = interpreter.get_output_details()
print(str(output_details))

[{'name': 'sub_2', 'index': 97, 'shape': array([  1, 353, 257,   3], dtype=int32), 'shape_signature': array([  1, 353, 257,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'float_heatmaps', 'index': 93, 'shape': array([ 1, 23, 17, 17], dtype=int32), 'shape_signature': array([ 1, 23, 17, 17], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'float_short_offsets', 'index': 96, 'shape': array([ 1, 23, 17, 34], dtype=int32), 'shape_signature': array([ 1, 23, 17, 34], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'z

In [4]:
# check outputstrids:  resolution = ((InputImageSize - 1) / OutputStride) + 1

output_stridex = (input_details[0]['shape'][1]-1)/(output_details[0]['shape'][1]-1)
output_stridey = (input_details[0]['shape'][2]-1)/(output_details[0]['shape'][2]-1)
print("outputstrides:", output_stridex, output_stridey)

outputstrides: 16.0 16.0


In [5]:
output_stride = 16.0
num_keypoints = 17

In [6]:
interpreter.allocate_tensors()

In [7]:
imgpath = "./test.png"

In [8]:
timg = Image.open(imgpath)
timg2 = timg.resize((257,353))
tim = np.array(timg2)
tim2 = np.expand_dims(tim, axis=0)

In [9]:
t1 = time.time()
interpreter.set_tensor(input_details[0]['index'], tim2.astype(np.float32))
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
t2 = time.time()
print("time:",t2-t1)

time: 0.1250917911529541


In [10]:
heatmaps = np.squeeze(interpreter.get_tensor(output_details[0]['index']))
offsets = np.squeeze(interpreter.get_tensor(output_details[1]['index']))

In [11]:
heatmaps.shape, offsets.shape

((23, 17, 17), (23, 17, 34))

In [12]:
def get_keypoints(heatmaps, offsets, output_stride=16.0):
#     scores = np.sigmoid(heatmaps)
    scores = heatmaps
    num_keypoints = scores.shape[2]
    heatmap_positions = []
    offset_vectors = []
    confidences = []
    for ki in range(0, num_keypoints ):
        x,y = np.unravel_index(np.argmax(scores[:,:,ki]), scores[:,:,ki].shape)
        confidences.append(scores[x,y,ki])
        offset_vector = (offsets[x,y,ki], offsets[x,y,num_keypoints+ki])
        heatmap_positions.append((x,y))
        offset_vectors.append(offset_vector)
    image_positions = np.add(np.array(heatmap_positions) * output_stride, offset_vectors)
    keypoints = [(i, pos, confidences[i]) for i, pos in enumerate(image_positions)]
    return keypoints

In [13]:
get_keypoints(heatmaps, offsets)

[(0, array([354.82839108, 131.42807031]), -6.001495),
 (1, array([354.04453778, 129.9302243 ]), -6.3138146),
 (2, array([344.65926647,  97.75874472]), -5.739022),
 (3, array([331.14421463, 133.42482138]), -5.1812615),
 (4, array([329.15613747, 146.1574614 ]), -5.296742),
 (5, array([348.49900341, 101.26444244]), -5.2257323),
 (6, array([363.31262207, 127.49399906]), -4.996435),
 (7, array([341.60149193, 146.26859498]), -6.755512),
 (8, array([341.91810608, 112.85078174]), -6.5640254),
 (9, array([325.68569613,  -4.64089632]), -6.5551777),
 (10, array([297.60314655,  -5.32451296]), -6.7565513),
 (11, array([ 0.46389624, 33.89114761]), -6.184896),
 (12, array([ 1.76255298, 33.14163578]), -6.372113),
 (13, array([321.72717941, 263.08027792]), -6.5485115),
 (14, array([321.22483706, 262.17429972]), -7.002337),
 (15, array([324.96882343,  -5.27495909]), -6.779457),
 (16, array([310.54090023,  -6.91213655]), -6.6521845)]

In [49]:
for i in range(num_keypoints):
    x,y = np.unravel_index(np.argmax(heatmaps[:,:,i]), heatmaps[:,:,i].shape) 
    conf = heatmaps[x,y,i]
    offset_vector = (offsets[x,y,i], offsets[x,y,num_keypoints+i])
    print(x,y,conf, offset_vector)

    pos_i = np.add(np.array([x,y]) * output_stride, offset_vector)
    print(pos_i)

20 15 -6.1496406 (5.4461565, 10.782186)
[325.4461565  250.78218555]
22 14 -6.304964 (2.15078, -1.1065224)
[354.15077996 222.89347756]
20 15 -5.4582505 (1.8755507, 9.685787)
[321.87555075 249.6857872 ]
19 0 -6.501217 (1.3178866, -2.7529018)
[305.31788659  -2.75290179]
20 15 -5.8539906 (5.0117784, 6.784121)
[325.01177835 246.78412104]
22 6 -5.4797087 (7.13035, 3.732857)
[359.13035011  99.73285699]
22 4 -5.6351075 (8.20594, -1.8878671)
[360.20594025  62.11213291]
2 12 -6.895049 (5.8851876, 18.928425)
[ 37.88518763 210.92842484]
20 15 -6.560242 (11.956549, 2.4038842)
[331.95654869 242.40388417]
19 0 -6.995436 (3.2525666, -7.669128)
[307.25256658  -7.66912794]
18 0 -6.7330246 (6.677606, -7.450439)
[294.67760611  -7.45043898]
22 2 -6.204076 (2.7541113, 1.2149421)
[354.75411129  33.2149421 ]
22 3 -5.9366326 (6.1759667, 1.675064)
[358.17596674  49.67506397]
20 16 -7.461457 (4.255873, 4.5832915)
[324.2558732  260.58329153]
22 16 -7.1741033 (1.3165295, 2.5614555)
[353.31652951 258.56145549]
22 6

In [ ]:
resimg = visual("/home/cp/Desktop/test1.png", kps)

In [ ]:
resimg

In [ ]:
def extract_keypoints(resdata):
    if len(resdata.shape)==4:
        resdata = resdata[0]
    kps = []
    for i in range(14):
        tmpdata = resdata[:,:,i]
        pos = np.unravel_index(np.argmax(tmpdata),tmpdata.shape)
        kps.append(pos)
    return kps

In [ ]:
def visual(imgpath, kps):
    timg = Image.open(imgpath)
    w,h = timg.size
    draw = ImageDraw.Draw(timg)
    for row, col in kps:
        row = int(row/17.0*w)
        col = int(col/23.0*h)
        draw.arc((col-2,row-2, col+2, row+2), 0,360, fill='red', width=1)
    return timg